# 3th attempt - NN

In [1]:
import numpy as np
import pandas as pd
from functions import *
from read_from_file_df import *
import pickle
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [ ]:
SIZE = 5
AMOUNT_BOARDS = 10000
AMOUNT_MOVES = 100
NUM_DICT = 1

In [ ]:
gen = 2
name_df = f'{PATH_DF}\\{SIZE}-{AMOUNT_BOARDS}\\{SIZE}size_{AMOUNT_BOARDS}boards_{gen}gen_reverse'reverse_df = pd.read_pickle(f'{name_df}.pkl')

In [ ]:
new_columns = [f'Col_{i}' for i in range(gen*SIZE*SIZE)]
reverse_df_sort = reverse_df.sort_values(by = new_columns).reset_index(drop=True)
for i in reverse_df_sort.columns:
    reverse_df_sort[i] = reverse_df_sort[i].astype(int)

In [5]:
print("reverse df:", len(reverse_df))
print("reverse df sort:",len(reverse_df_sort))

reverse df: 162937
reverse df sort: 162937


In [6]:
# Step 1: Prepare Data
amount_features = len(reverse_df_sort.columns) - SIZE*SIZE #the previous boards
features = reverse_df_sort.iloc[:, :amount_features]
name_col = 'Col_' + str(amount_features + 1)  # Target: the first pixel in the board
target = reverse_df_sort[name_col]

# Step 2: Split Data
X_train_val, X_test, y_train_val, y_test = train_test_split(features, target, test_size=0.1, random_state=365)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.1, random_state=365)

print("len x train: ", len(X_train))
print("len x test: ",len(X_test))
print("len y train: ",len(y_train))
print("len y test: ",len(y_test))

len x train:  131978
len x test:  16294
len y train:  131978
len y test:  16294


In [7]:
X_train.shape

(131978, 25)

In [8]:
X_train_array = X_train.to_numpy()
y_train_array = y_train.to_numpy()

In [9]:
print(X_train_array.shape)
print(y_train_array.shape)

(131978, 25)
(131978,)


In [10]:
X_train_array = X_train_array.reshape((X_train.shape[0],SIZE,SIZE,1))
y_train_array = y_train_array.reshape((y_train.shape[0],1))

In [11]:
print(X_train_array.shape)
print(y_train_array.shape)

(131978, 5, 5, 1)
(131978, 1)


In [12]:
X_val_array = X_val.to_numpy()
X_val_array = X_val_array.reshape((X_val.shape[0],SIZE,SIZE,1))
y_val_array = y_val.to_numpy()
y_val_array = y_val_array.reshape((y_val.shape[0],1))

X_test_array = X_test.to_numpy()
X_test_array = X_test_array.reshape((X_test.shape[0],SIZE,SIZE,1))
y_test_array = y_test.to_numpy()
y_test_array = y_test_array.reshape((y_test.shape[0],1))

In [13]:
import tensorflow as tf

input_shape = (SIZE, SIZE, 1)

model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=input_shape),

    tf.keras.layers.Flatten(),

    tf.keras.layers.Dense(128, activation='relu'),  

    tf.keras.layers.Dense(64, activation='relu'),

    tf.keras.layers.Dense(1, activation='sigmoid')  
])

model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten (Flatten)               │ (None, 25)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │         3,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,649 (45.50 KB)

 Trainable params: 11,649 (45.50 KB)

 Non-trainable params: 0 (0.00 B)

In [14]:
model.compile(optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy'])

In [15]:
history = model.fit(X_train_array, y_train_array, 
    validation_data=(X_val_array, y_val_array),
    epochs=10,
    batch_size=32)


Epoch 1/10
4125/4125 ━━━━━━━━━━━━━━━━━━━━ 22s 5ms/step - accuracy: 0.6630 - loss: 0.6162 - val_accuracy: 0.6719 - val_loss: 0.6038
Epoch 2/10
4125/4125 ━━━━━━━━━━━━━━━━━━━━ 27s 6ms/step - accuracy: 0.6847 - loss: 0.5939 - val_accuracy: 0.6841 - val_loss: 0.5940
Epoch 3/10
4125/4125 ━━━━━━━━━━━━━━━━━━━━ 24s 6ms/step - accuracy: 0.6925 - loss: 0.5841 - val_accuracy: 0.6865 - val_loss: 0.5890
Epoch 4/10
4125/4125 ━━━━━━━━━━━━━━━━━━━━ 30s 3ms/step - accuracy: 0.6987 - loss: 0.5744 - val_accuracy: 0.6897 - val_loss: 0.5869
Epoch 5/10
4125/4125 ━━━━━━━━━━━━━━━━━━━━ 20s 3ms/step - accuracy: 0.7048 - loss: 0.5682 - val_accuracy: 0.6867 - val_loss: 0.5950
Epoch 6/10
4125/4125 ━━━━━━━━━━━━━━━━━━━━ 14s 3ms/step - accuracy: 0.7075 - loss: 0.5651 - val_accuracy: 0.6894 - val_loss: 0.5826
Epoch 7/10
4125/4125 ━━━━━━━━━━━━━━━━━━━━ 26s 5ms/step - accuracy: 0.7106 - loss: 0.5618 - val_accuracy: 0.6872 - val_loss: 0.5916
Epoch 8/10
4125/4125 ━━━━━━━━━━━━━━━━━━━━ 19s 4ms/step - accuracy: 0.7165 - loss: 0

In [16]:
test_loss, test_acc = model.evaluate(X_test_array, y_test_array)
print(f"Test accuracy: {test_acc:.3f}")

510/510 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6982 - loss: 0.5851
Test accuracy: 0.701
